In [1]:
import sqlite3

# Connect to a database (creates a new database if it doesn't exist!)
conn = sqlite3.connect('./databases/publications-database.db')

# Enable foreign key support
conn.execute("PRAGMA foreign_keys = ON")

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Create a table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Publication(
        publication_id INTEGER PRIMARY KEY,
        title TEXT,
        abstract TEXT,
        year TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Author(
        author_id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS written_by(
        publication_id INTEGER,
        author_id INTEGER,
        PRIMARY KEY (publication_id, author_id),
        FOREIGN KEY (publication_id) REFERENCES Publication(publication_id),
        FOREIGN KEY (author_id) REFERENCES Author(author_id)
    )
''')


# Save the changes and close the connection
conn.commit()
conn.close()

### The database is filled with the data from the database from another project (kool)

#### <span style="color:red">Note that this is only possible if the database has been newly created and is empty</span>

In [2]:
import sqlite3

# Connect to source and destination databases
source_conn = sqlite3.connect('./databases/database-kool.db')
destination_conn = sqlite3.connect('./databases/publications-database.db')

# Create cursors for both databases
source_cursor = source_conn.cursor()
destination_cursor = destination_conn.cursor()

# Create tables in destination database
# [Your provided table creation code goes here for the destination_cursor]

# Fetch and copy data from Publication table
source_cursor.execute('SELECT abstract_id, title, content, year FROM abstract')
for row in source_cursor.fetchall():
    destination_cursor.execute('''
        INSERT INTO Publication(publication_id, title, abstract, year) 
        VALUES (?, ?, ?, ?)''', row)

# Fetch and copy data from Author table
source_cursor.execute('SELECT author_id, first_name, last_name FROM author')
for row in source_cursor.fetchall():
    destination_cursor.execute('''
        INSERT INTO Author(author_id, first_name, last_name) 
        VALUES (?, ?, ?)''', row)

# Fetch and copy data from written_by table
source_cursor.execute('SELECT abstract_id, author_id FROM written_by')
for row in source_cursor.fetchall():
    destination_cursor.execute('''
        INSERT INTO written_by(publication_id, author_id) 
        VALUES (?, ?)''', row)

# Commit the transactions and close the connections
destination_conn.commit()
source_conn.close()
destination_conn.close()

### Resetting the publication_ids because they were not ordered correctly

In [ ]:
# Connect to a database
conn = sqlite3.connect('./databases/publications-database.db')
cursor = conn.cursor()

# Step 1: Create a mapping of old publication_id values to the new ones
cursor.execute("SELECT publication_id FROM Publication")
all_ids = [row[0] for row in cursor.fetchall()]

id_mapping = {}
for new_id, old_id in enumerate(all_ids, start=0):
    id_mapping[old_id] = new_id

# Step 2: Update the written_by table using the mapping
for old_id, new_id in id_mapping.items():
    cursor.execute("UPDATE written_by SET publication_id = ? WHERE publication_id = ?", (new_id, old_id))

# Step 3: Update the Publication table with the new publication_id values
for old_id, new_id in id_mapping.items():
    cursor.execute("UPDATE Publication SET publication_id = ? WHERE publication_id = ?", (new_id, old_id))

# Commit the changes and close the connection
conn.commit()
conn.close()